### Normal Chain

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)
template = PromptTemplate.from_template("You are a helpful assistant. Answer the question: {question}?")
parser = StrOutputParser()

chain = template | llm | parser

result = chain.invoke({"question": "What is the capital of France?"})
print(result)

The capital of France is Paris.


In [4]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


### Sequential Chain

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)
parser = StrOutputParser()

seq_template_1 = PromptTemplate.from_template("Give me a short summary of the topic: {topic}")

seq_template_2 = PromptTemplate.from_template("Based on the summary, give me 5 key points on it: {summary}")

seq_chain = seq_template_1 | llm | parser | seq_template_2 | llm | parser

result = seq_chain.invoke({"topic": "Cats are the best animal"})
print(result)


1. **Independent Nature**: Cats are low-maintenance pets due to their independent behavior, requiring less attention compared to dogs.

2. **Affectionate Bonds**: They are playful and affectionate, capable of forming strong emotional connections with their owners.

3. **Unique Personalities**: Cats exhibit distinctive behaviors, such as purring and kneading, which provide joy and entertainment to their owners.

4. **Cleanliness**: Cats are naturally clean animals, as they regularly groom themselves, making them easy to care for.

5. **Adaptability**: Their ability to thrive in various living environments, from small apartments to larger homes, makes them versatile companions.


In [6]:
seq_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

### Parallel Chain

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)
parser = StrOutputParser()

par_template_1 = PromptTemplate.from_template("Give me a short summary of the topic: {topic}")
par_template_2 = PromptTemplate.from_template("Based on the summary, give me 5 key points on it: {topic}")

par_chain_1 = par_template_1 | llm | parser
par_chain_2 = par_template_2 | llm | parser

parallel_chain = RunnableParallel({
    "summary": par_chain_1,
    "points": par_chain_2
}) 

mer_template = PromptTemplate.from_template("""
Combine the summary and points, and give a comprehensive output:

Summary: {summary}
Points: {points}
                                            
""")  

mer_chain = mer_template | llm | parser

final_chain = parallel_chain | mer_chain

result = final_chain.invoke({"topic": "Cats are the best animal"})
print(result)


Cats are often regarded as the best animals by many enthusiasts due to their unique combination of independence, affectionate behavior, and low maintenance requirements. Their playful antics and soothing purrs not only entertain but also foster strong bonds with humans, while they simultaneously enjoy their own space. This adaptability makes them suitable companions for a wide range of living environments and lifestyles.

Here are five key points that support the belief that cats are the best animals:

1. **Companionship**: Cats offer affectionate companionship and often develop strong emotional connections with their owners, enhancing overall emotional well-being.

2. **Low Maintenance**: Unlike many pets, cats are relatively low-maintenance. Their independent nature means they require less attention and care compared to dogs, making them ideal for busy individuals.

3. **Pest Control**: As natural hunters, cats effectively help control household pests, such as mice and insects, contr

In [9]:
final_chain.get_graph().print_ascii()

          +-------------------------------+          
          | Parallel<summary,points>Input |          
          +-------------------------------+          
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                    +------------+   
  | ChatOpenAI |                    | ChatOpenAI |   
  +------------+                    +------------+   
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

### Conditional Chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableBranch
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

class SentimentReview (BaseModel):
    sentiment: Literal['positive', 'negative', 'neutral'] = Field(description="The sentiment of the review")

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)
str_parser = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=SentimentReview)

template = PromptTemplate.from_template("""
What is the sentiment of the following review?

Review: {review}
Sentiment: {format_instructions}
""", 
partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

review_chain = template | llm | parser | pydantic_parser

pos_template = PromptTemplate.from_template("""
    The review is positive. So write a short appreciation: \n Review: {review}
""")

neg_template = PromptTemplate.from_template("""
    The review is negative. So write a short critic: \n Review: {review}
""")

neu_template = PromptTemplate.from_template("""
    The review is neutral. So write a short balanced remark: \n Review: {review}
""")

default_template = PromptTemplate.from_template("""
    I could not detect the sentiment. Provide a neutral response: \n Review: {review}
""")

branch_chain = RunnableBranch(
    (lambda x: x.sentiment == "positive", pos_template | llm | str_parser),
    (lambda x: x.sentiment == "negative", neg_template | llm | str_parser),
    (lambda x: x.sentiment == "neutral", neu_template | llm | str_parser),
    default_template | llm | str_parser
)



sentiment='positive'
